<a href="https://colab.research.google.com/github/Mustaq7777777/Introduction_To_Deep_Learning_DA6401/blob/main/Assignment1/DeepLearning_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install wandb

question 1

In [ ]:
import wandb
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist


#There are 10 class labels present in mnist data set.
class_labels = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]


wandb.init(project="DA6401-Assignment1", name="run1", reinit=True)

#Loading data from the fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

#storing images in Image_set
images_to_log = []
for class_index in range(10):

    #Find the instance of each class and append it .
    idx = np.where(train_labels == class_index)[0][0]


    image = train_images[idx]
    wandb_image = wandb.Image(image, caption=class_labels[class_index])
    images_to_log.append(wandb_image)

# Log the image to WandB
wandb.log({"Question1 Sample images for each class": images_to_log})

wandb.finish()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist

Activation Functions and their derivatives

In [ ]:

def softmax(v):
    exp_vector = np.exp(v - np.max(v))
    return exp_vector / np.sum(exp_vector)

# sigmoid

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def d_sigmoid(x):
    return sigmoid(x) * (1 - sigmoid(x))

# tanh

def tanh(x):
    return np.tanh(x)

def d_tanh(x):
    return 1 - np.tanh(x) ** 2

# relu

def relu(x):
    return np.maximum(0, x)

def d_relu(x):
    if x > 0:
      return 1
    else:
      return 0

# one hot vector
def e(l, len):
    y = np.zeros([len, 1])
    y[l] = 1
    return y


Initialization Of Weights and Biases

In [ ]:

def init(W, b, input_nodes, hiddenlayers, hiddennodes, output_nodes, initializer):

    if initializer == "Xavier":
        # Xavier initialization for the first hidden layer (input to first hidden layer)
        W[1] = np.random.normal(0.0, np.sqrt(1.0 / input_nodes), (hiddennodes, input_nodes))
        b[1] = np.zeros((hiddennodes, 1))

        # Xavier initialization for subsequent hidden layers
        for i in range(2, hiddenlayers + 1):
            W[i] = np.random.normal(0.0, np.sqrt(1.0 / hiddennodes), (hiddennodes, hiddennodes))
            b[i] = np.zeros((hiddennodes, 1))

        # Xavier initialization for the output layer (from last hidden layer to output)
        W[hiddenlayers + 1] = np.random.normal(0.0, np.sqrt(1.0 / hiddennodes), (output_nodes, hiddennodes))
        b[hiddenlayers + 1] = np.zeros((output_nodes, 1))

    elif initializer == "random":
        # Random initialization for the first hidden layer (input to first hidden layer)
        W[1] = np.random.rand(hiddennodes, input_nodes) - 0.5
        b[1] = np.zeros((hiddennodes, 1))

        # Random initialization for subsequent hidden layers
        for i in range(2, hiddenlayers + 1):
            W[i] = np.random.rand(hiddennodes, hiddennodes) - 0.5
            b[i] = np.zeros((hiddennodes, 1))

        # Random initialization for the output layer (from last hidden layer to output)
        W[hiddenlayers + 1] = np.random.rand(output_nodes, hiddennodes) - 0.5
        b[hiddenlayers + 1] = np.zeros((output_nodes, 1))


    return W, b



Forward Propagation

In [ ]:
# for applying activation function
def g(z, act_func):
    if act_func == "sigmoid":
        return sigmoid(z)
    elif act_func == "tanh":
        return tanh(z)
    elif act_func == "relu":
        return relu(z)

# for applying derivative of activation function
def g_prime(z, act_func):
    if act_func == "sigmoid":
        return d_sigmoid(z)
    elif act_func == "tanh":
        return d_tanh(z)
    elif act_func == "relu":
        return d_relu(z)

# for calculating output
def o(z, act_func):
    if act_func == "softmax":
        return softmax(z)

# Applying forward propagation for the (L-1) th training example
def forward_propagation(x_train, hidden_layers, hidden_nodes, input_nodes, output_nodes, W, b, a, h, L, act_func):
    h[0] = x_train[L].reshape(-1, 1)
    for layer in range(1, hidden_layers + 1):
        a[layer] = np.dot(W[layer], h[layer - 1]) + b[layer]
        h[layer] = g(a[layer], act_func)
    a[hidden_layers + 1] = np.dot(W[hidden_layers + 1], h[hidden_layers]) + b[hidden_layers + 1]
    ycap = o(a[hidden_layers + 1], "softmax")
    return ycap, a, h


Back Propagation

In [ ]:
def back_prop(y_train, hiddenlayers, hiddennodes, input_nodes, output_nodes,
              W, b, a, h, VW, Vb, sample_idx, yhat, activation_func, loss_func):

    output_layer = hiddenlayers + 1
    delta = {}

    if loss_func == "cross_entropy":
        delta[output_layer] = yhat - e(y_train[sample_idx], output_nodes)

    for k in range(output_layer, 0, -1):
        grad_W = np.dot(delta[k], h[k-1].T)
        grad_b = delta[k]
        if k in VW:
            VW[k] += grad_W
            Vb[k] += grad_b
        else:
            VW[k] = grad_W
            Vb[k] = grad_b
        if k > 1:
            prev_error = np.dot(W[k].T, delta[k])
            delta[k-1] = np.multiply(prev_error, g_pr(a[k-1], hiddennodes, activation_func).reshape(-1, 1))

    return VW, Vb


# loss function (currently for cross-entropy and squared_error)
def loss(yhat, y_train, loss_func, sample_idx, output_nodes):
    if loss_func == "cross_entropy":
        return -np.log(yhat[y_train[sample_idx]])
    elif loss_func == "squared_error":
        return np.square(yhat - e(y_train[sample_idx], output_nodes))